In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import scipy
import pickle

In [14]:
# Read in Full Data Set that Jonathan created
stock_df = pd.read_csv('../../data/all_stocks.csv')
stock_df = stock_df.set_index('Date')

In [31]:
# Load Jonathan's dict of stocks for each year
with open('../../data/valid_stocks_by_year.txt', 'rb') as f:
    valid_years= pickle.load(f)

In [6]:
sys.path.append('../../source/')
from JungeUtilities import beta_hedging, portfolio_ize, date_to_int
from get_beta import get_beta

In [42]:
year = 2010

In [43]:
year_symbols = list(valid_years[2010])

In [44]:
portfolio = list(np.random.choice(year_symbols, 20))

In [45]:
portfolio_df = stock_df[portfolio][(stock_df[portfolio].index > str(year-1)+'-12-31') & \
                                   (stock_df[portfolio].index < str(year+1)+'-01-01')]

In [47]:
portfolio_df.head()

,AEO,SEED,AMOT,GHI,FDI,PXLW,TISI,ARMH,MVF,FOE,CECE,RHP,CHCI,EVC,AUMN,NPI,BKS,AI,DRI,BT
Date,,,,,,,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,12.15,14.48,NaN,NaN,8.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.15,NaN,NaN,NaN,21.74
2010-01-04,12.845210,12.23,2.400675,12.40,14.44,3.00,19.299999,8.85,6.039345,8.61,3.630903,14.637174,6.159994,2.900934,10.25,13.13,11.433103,7.477629,27.900050,11.13
2010-01-05,13.252038,12.96,2.476887,12.41,14.45,3.08,18.910000,8.93,6.032830,8.64,3.603671,14.538227,6.299994,2.800032,11.50,13.12,11.480767,7.634204,27.780477,11.32
2010-01-06,13.229436,12.51,2.629311,12.34,14.47,3.04,19.320000,9.12,6.039345,8.60,3.594594,14.962288,6.019994,2.657088,11.95,13.13,11.433103,7.629459,27.611868,11.56
2010-01-07,12.822609,12.05,2.619784,12.36,14.44,3.02,20.750000,9.37,6.091464,8.77,3.776139,14.948153,6.579993,2.825257,12.50,13.17,11.141170,7.615225,27.868797,11.36


In [68]:
def swap(market_symbols, year_symbols):
    """
    Randomly changes the symbols in a market. Can either grow or shrink the market by 1 asset.  
    
    Inputs:
    -------
    market_symbols: list of symbols that are in a market
    year_symbols:   list of all the possible symbols for a given year
    
    Output:
    -------
    symbols: list of symbols in the market after the random change.  
    
    """
    coin_flip = np.random.binomial(1,0.5)
    
    if coin_flip==0: # Shrink the market by one asset
        
        # Pick a random list index and pop that symbol off
        market_symbols.pop(np.random.randint(len(market_symbols)))
        return market_symbols
    else: # Grow the market by one asset
        
        # Get a list of the symbols for that year that are not already in the portfolio
        potential_symbols = list(set(year_symbols) - set(market_symbols))
        symbol_to_add = np.random.choice(potential_symbols)
        market_symbols.append(symbol_to_add)
        return market_symbols

In [78]:
def simulated_annealing(portfolio, initial_market, init_temp, min_temp, cool_by, reanneal, num_iter, energy_func):
    """
    Inputs:
    -------
    portfolio:
        List of symbols in the portfolio
    starting_market:
        List of symbols in the initial market basket.  
    init_temp: 
        float that controls the initial temperature of the algorithm
    min_temp: 
        float that acts as a floor for the decreasing temperature.  When this is hit, we heat back up to init_temp.  
    cool_by: 
        float that controls the speed that the cooling occurs
    reanneal: 
        integer that controls how many iterations pass between cooling steps
    num_iter: 
        integer that controls the total number of iterations that the algorithm runs
    energy_func:
        function that determines the energy state, eg correlation, volatility, sharpe ratio
    
    Returns:
    --------
    states: 
        a history of the current energy state at each iteration
    best_market: 
        list of the symbols for the best hedging market found
    """
    
    market = initial_market
    
    # A running account of the best market found.  This is updated as better markets are found.
    
    #####   1) get df for market, portfolio. 
            2) portfolio_ize using weights
            3) get_betas
            4) beta_hedge
            5) energy_func on hedged returns
            
    best_market_energy = energy_func(____)
    best_market = initial_market
    
    
    # Initial value for old_E is the initial total value of the starting point.
    old_E = best_market_energy
    
    temperature = init_temp
    
    # A history of the current state of the algorithm
    states = np.zeros(num_iter)
    
    for i in range(num_iter):
        
        # Switch the bag up a little bit
        proposed_market = swap(market)
        
        #   1) get df for portfolio
            2) portfolio_ize using weights
            3) get_betas
            4) beta_hedge
            5) hedged_returns

        # Examine energy state of the new bag
        new_E = energy_func(hedged_returns)
        delta_E = new_E - old_E
        
        # We always accept an improvement
        if new_E < old_E:
            market = proposed_market
            states[i] = new_E
            old_E = new_E
            # Update our running best bag found
            if new_E > best_market_energy:
                best_market_energy = new_E
                best_market = market
        # We sometimes accept a decline because this can get us out of a local maximum
        elif np.random.rand() < np.exp(-delta_E/temperature):
            market = proposed_proposed
            states[i] = new_E
            old_E = new_E

        # And sometimes we just stay where we are until something better comes along
        else: 
            states[i] = old_E
            
        # Cool down slowly at the defined interval
        if num_iter % reanneal == 0:
            temperature = temperature * cool_by
            
            # Reheat when the temperature gets too cold
            if temperature < min_temp:
                temperature = init_temp
                
    return states, best_markets

IndentationError: unexpected indent (<ipython-input-78-cbabe10a1fb8>, line 35)